<a href="https://colab.research.google.com/github/rashida048/hypermeter-tuning/blob/main/hyperparameterTuning_with_kerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/hyperparameter-keras-tuner/hyperparameter-keras-tuner.zip
!unzip -qq hyperparameter-keras-tuner.zip
%cd hyperparameter-keras-tuner

--2023-06-06 02:21:50--  https://pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com/hyperparameter-keras-tuner/hyperparameter-keras-tuner.zip
Resolving pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)... 52.92.147.178, 3.5.79.21, 52.92.128.58, ...
Connecting to pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com (pyimagesearch-code-downloads.s3-us-west-2.amazonaws.com)|52.92.147.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142561 (139K) [binary/octet-stream]
Saving to: ‘hyperparameter-keras-tuner.zip’

hyperparameter-kera 100%[===================>] 139.22K   570KB/s    in 0.2s    

2023-06-06 02:21:51 (570 KB/s) - ‘hyperparameter-keras-tuner.zip’ saved [142561/142561]

/content/hyperparameter-keras-tuner


In [2]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.7 MB/s eta 0:00:00


In [3]:
import matplotlib
matplotlib.use("Agg")

# import the necessary package
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import keras_tuner as kt
import numpy as np
import argparse
import cv2

In [4]:
class Config:
    # define the path to our output directory
    OUTPUT_PATH = "output"

    # initialize the input shape and number of classes
    INPUT_SHAPE = (28, 28, 1)
    NUM_CLASSES = 10

    # define the total number of epochs to train, batch size, and the
    # early stopping patience
    EPOCHS = 50
    BS = 32
    EARLY_STOPPING_PATIENCE = 5

# instantiate an object of the configuration class
config = Config()

In [5]:
def save_plot(H, path):
	# plot the training loss and accuracy
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(H.history["loss"], label="train_loss")
	plt.plot(H.history["val_loss"], label="val_loss")
	plt.plot(H.history["accuracy"], label="train_acc")
	plt.plot(H.history["val_accuracy"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend()
	plt.savefig(path)

In [6]:
def build_model(hp):
	# initialize the model along with the input shape and channel
	# dimension
	model = Sequential()
	inputShape = config.INPUT_SHAPE
	chanDim = -1

	# first CONV => RELU => POOL layer set
	model.add(Conv2D(
		hp.Int("conv_1", min_value=32, max_value=96, step=32),
		(3, 3), padding="same", input_shape=inputShape))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	# second CONV => RELU => POOL layer set
	model.add(Conv2D(
		hp.Int("conv_2", min_value=64, max_value=128, step=32),
		(3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))

	# first (and only) set of FC => RELU layers
	model.add(Flatten())
	model.add(Dense(hp.Int("dense_units", min_value=256,
		max_value=768, step=256)))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))

	# softmax classifier
	model.add(Dense(config.NUM_CLASSES))
	model.add(Activation("softmax"))

	# initialize the learning rate choices and optimizer
	lr = hp.Choice("learning_rate",
		values=[1e-1, 1e-2, 1e-3])
	opt = Adam(learning_rate=lr)

	# compile the model
	model.compile(optimizer=opt, loss="categorical_crossentropy",
		metrics=["accuracy"])

	# return the model
	return model

In [7]:
args = {
	"tuner": "random",
	"plot": "output/random_plot.png"
}

In [8]:
print("[INFO] loading Fashion MNIST...")
((trainX, trainY), (testX, testY)) = fashion_mnist.load_data()

# add a channel dimension to the dataset
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

# scale data to the range of [0, 1]
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0

# one-hot encode the training and testing labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

# initialize the label names
labelNames = ["top", "trouser", "pullover", "dress", "coat",
	"sandal", "shirt", "sneaker", "bag", "ankle boot"]

[INFO] loading Fashion MNIST...
4422102/4422102 [==============================] - 0s 0us/step


In [9]:
es = EarlyStopping(
	monitor="val_loss",
	patience=config.EARLY_STOPPING_PATIENCE,
	restore_best_weights=True)

In [10]:
# check if we will be using the hyperband tuner
if args["tuner"] == "hyperband":
	# instantiate the hyperband tuner object
	print("[INFO] instantiating a hyperband tuner object...")
	tuner = kt.Hyperband(
		build_model,
		objective="val_accuracy",
		max_epochs=config.EPOCHS,
		factor=3,
		seed=42,
		directory=config.OUTPUT_PATH,
		project_name=args["tuner"])

# check if we will be using the random search tuner
elif args["tuner"] == "random":
	# instantiate the random search tuner object
	print("[INFO] instantiating a random search tuner object...")
	tuner = kt.RandomSearch(
		build_model,
		objective="val_accuracy",
		max_trials=10,
		seed=42,
		directory=config.OUTPUT_PATH,
		project_name=args["tuner"])

# otherwise, we will be using the bayesian optimization tuner
else:
	# instantiate the bayesian optimization tuner object
	print("[INFO] instantiating a bayesian optimization tuner object...")
	tuner = kt.BayesianOptimization(
		build_model,
		objective="val_accuracy",
		max_trials=10,
		seed=42,
		directory=config.OUTPUT_PATH,
		project_name=args["tuner"])

[INFO] instantiating a random search tuner object...


In [11]:
# perform the hyperparameter search
print("[INFO] performing hyperparameter search...")
tuner.search(
	x=trainX, y=trainY,
	validation_data=(testX, testY),
	batch_size=config.BS,
	callbacks=[es],
	epochs=config.EPOCHS
)

Trial 10 Complete [00h 02m 40s]
val_accuracy: 0.9168999791145325

Best val_accuracy So Far: 0.9269000291824341
Total elapsed time: 00h 18m 55s


In [12]:
bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(
	bestHP.get("conv_1")))
print("[INFO] optimal number of filters in conv_2 layer: {}".format(
	bestHP.get("conv_2")))
print("[INFO] optimal number of units in dense layer: {}".format(
	bestHP.get("dense_units")))
print("[INFO] optimal learning rate: {:.4f}".format(
	bestHP.get("learning_rate")))

[INFO] optimal number of filters in conv_1 layer: 96
[INFO] optimal number of filters in conv_2 layer: 96
[INFO] optimal number of units in dense layer: 512
[INFO] optimal learning rate: 0.0010


In [13]:
model = tuner.hypermodel.build(bestHP)
H = model.fit(x=trainX, y=trainY,
	validation_data=(testX, testY), batch_size=config.BS,
	epochs=config.EPOCHS, callbacks=[es], verbose=1)


Epoch 1/50
1875/1875 [==============================] - 14s 6ms/step - loss: 0.4171 - accuracy: 0.8566 - val_loss: 0.3374 - val_accuracy: 0.8706
Epoch 2/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2842 - accuracy: 0.8975 - val_loss: 0.3163 - val_accuracy: 0.8791
Epoch 3/50
1875/1875 [==============================] - 10s 6ms/step - loss: 0.2546 - accuracy: 0.9078 - val_loss: 0.3167 - val_accuracy: 0.8915
Epoch 4/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2233 - accuracy: 0.9187 - val_loss: 0.2417 - val_accuracy: 0.9136
Epoch 5/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.2117 - accuracy: 0.9244 - val_loss: 0.2484 - val_accuracy: 0.9150
Epoch 6/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1898 - accuracy: 0.9320 - val_loss: 0.2614 - val_accuracy: 0.9100
Epoch 7/50
1875/1875 [==============================] - 11s 6ms/step - loss: 0.1788 - accuracy: 0.9349 - val_loss: 0.3418 - val_ac

In [14]:
predictions = model.predict(x=testX, batch_size=32)
print(classification_report(testY.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames))

# generate the training loss/accuracy plot
save_plot(H, args["plot"])

313/313 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

         top       0.90      0.85      0.87      1000
     trouser       1.00      0.98      0.99      1000
    pullover       0.85      0.89      0.87      1000
       dress       0.88      0.95      0.92      1000
        coat       0.91      0.77      0.83      1000
      sandal       0.98      0.98      0.98      1000
       shirt       0.71      0.80      0.75      1000
     sneaker       0.96      0.97      0.96      1000
         bag       0.99      0.98      0.98      1000
  ankle boot       0.97      0.97      0.97      1000

    accuracy                           0.91     10000
   macro avg       0.92      0.91      0.91     10000
weighted avg       0.92      0.91      0.91     10000

